# Profiling ANDES in Notebook

## Profiling with Python CProfiler

Before getting started, this example requires the config flag `PFlow.init_tds` to be `0`, which is the default value.

In [1]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

Passing `profile=True, no_output = True` to `run` will enable the profiler and have the results printed.

In [2]:
ss = andes.run(case_path, profile=True, routine='tds', no_output=True)

Working directory: "/home/hcui7/repos/andes/examples"


Loaded config from file "/home/hcui7/.andes/andes.rc"


Loaded generated Python code in "~/.andes/pycode".


Parsing input file "/home/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.5330 seconds.


...

 96%|███████████████████████████████▋ | 96/100 [00:02<00:00, 25.96%/s]

100%|████████████████████████████████| 100/100 [00:02<00:00, 34.58%/s]

Simulation completed in 2.8943 seconds.



         1852891 function calls (1838465 primitive calls) in 3.840 seconds

   Ordered by: cumulative time
   List reduced from 4776 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.011    0.011    2.951    2.951 /home/hcui7/repos/andes/andes/routines/tds.py:248(run)
      603    0.155    0.000    2.681    0.004 /home/hcui7/repos/andes/andes/routines/tds.py:382(itm_step)


     2323    0.035    0.000    2.082    0.001 /home/hcui7/repos/andes/andes/routines/tds.py:797(fg_update)
    11674    0.163    0.000    1.969    0.000 /home/hcui7/repos/andes/andes/system.py:1525(call_models)
     2328    0.003    0.000    1.181    0.001 /home/hcui7/repos/andes/andes/system.py:892(g_update)
    23260    0.257    0.000    1.133    0.000 /home/hcui7/repos/andes/andes/core/model.py:1225(g_update)
        1    0.000    0.000    0.868    0.868 /home/hcui7/repos/andes/andes/main.py:268(load)
  855/172    0.003    0.000    0.540    0.003 <frozen importlib._bootstrap>:1002(_find_and_load)
  791/109    0.002    0.000    0.537    0.005 <frozen importlib._bootstrap>:967(_find_and_load_unlocked)
   693/30    0.001    0.000    0.536    0.018 <frozen importlib._bootstrap_external>:844(exec_module)
        1    0.000    0.000    0.534    0.534 /home/hcui7/repos/andes/andes/io/__init__.py:98(parse)
   709/31    0.001    0.000    0.533    0.017 {built-in method builtins.exec}
      

## Profiling with `line_profiler`.

`line_profiler` provides line-based profiling results for functions. 

Install with `pip install line_profiler` and restart the notebook.

In [3]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

### Profile power flow 

Pass the function name to profile to the magic `%lprun`, followed by a call to the function itself or an upper-level function.

Results will be shown in a popup window.

In [4]:
%load_ext line_profiler

%lprun -f andes.routines.pflow.PFlow.run andes.run(case_path, no_output=True, default_config=True)

Working directory: "/home/hcui7/repos/andes/examples"


Loaded generated Python code in "~/.andes/pycode".


Parsing input file "/home/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.1856 seconds.


System internal structure set up in 0.0638 seconds.


...

2: |F(x)| = 0.1701107882


3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103977e-07


Converged in 5 iterations in 0.0139 seconds.


-> Single process finished in 0.6131 seconds.


Alternatively, do

In [5]:
ss = andes.run(case_path, no_output=True, default_config=True)

Working directory: "/home/hcui7/repos/andes/examples"


Loaded generated Python code in "~/.andes/pycode".


Parsing input file "/home/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0772 seconds.


System internal structure set up in 0.0321 seconds.


...

2: |F(x)| = 0.1701107882


3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103977e-07


Converged in 5 iterations in 0.0100 seconds.


-> Single process finished in 0.2206 seconds.


In [6]:
ss.reset()
%lprun -f ss.PFlow.run ss.PFlow.run()

System internal structure set up in 0.0285 seconds.


-> System connectivity check results:


  No islanded bus detected.


  A total of 1 island(s) detected.


  Each island has a slack bus correctly defined and enabled.


...

1: |F(x)| = 3.608627841


2: |F(x)| = 0.1701107882


3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103977e-07


Converged in 5 iterations in 0.0129 seconds.


To dig into the Newton Raphson iteration steps, profile each step instead with:

In [7]:
ss.reset()
%lprun -f ss.PFlow.nr_step ss.PFlow.run()

System internal structure set up in 0.0319 seconds.


-> System connectivity check results:


  No islanded bus detected.


  A total of 1 island(s) detected.


  Each island has a slack bus correctly defined and enabled.


...

1: |F(x)| = 3.608627841


2: |F(x)| = 0.1701107882


3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103977e-07


Converged in 5 iterations in 0.0129 seconds.


### Profile time-domain simulation

In [8]:
%lprun -f ss.TDS.itm_step ss.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0-20.0 s.
Fixed step size: h=33.33 ms. Shrink if not converged.


Initialization for dynamics was successful in 0.0505 seconds.


  0%|                                          | 0/100 [00:00<?, ?%/s]

<Toggler 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
 10%|███▏                            | 10/100 [00:00<00:00, 110.24%/s]

 11%|███▋                             | 11/100 [00:00<00:00, 94.19%/s]

...

 86%|████████████████████████████▍    | 86/100 [00:02<00:00, 45.15%/s]

 92%|██████████████████████████████▎  | 92/100 [00:02<00:00, 48.49%/s]

 97%|████████████████████████████████ | 97/100 [00:02<00:00, 44.49%/s]

100%|████████████████████████████████| 100/100 [00:02<00:00, 35.25%/s]

Simulation completed in 2.8383 seconds.


## Cleanup

In [9]:
!andes misc -C


    _           _         | Version 1.4.2.post116.dev0+g832274a7
   /_\  _ _  __| |___ ___ | Python 3.9.6 on Linux, 09/26/2021 05:42:00 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

"/home/hcui7/repos/andes/examples/ieee14_out.lst" removed.
"/home/hcui7/repos/andes/examples/ieee14_out.txt" removed.
"/home/hcui7/repos/andes/examples/kundur_full_out.txt" removed.
"/home/hcui7/repos/andes/examples/kundur_full_out.npz" removed.
"/home/hcui7/repos/andes/examples/ieee14_out.npz" removed.
"/home/hcui7/repos/andes/examples/kundur_full_out.lst" removed.
